In [1]:
from selenium import webdriver
driver = webdriver.Chrome(executable_path = 'C:/Users/Sahil/.wdm/drivers/chromedriver/win32/110.0.5481/chromedriver.exe')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="9f167345258fa039c0900e62e9c65854")>

In [8]:
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import re

In [14]:
class Vaxijen_Antigenicity:
    def __init__(self):
        self.base_url = 'http://www.ddg-pharmfac.net/vaxijen/VaxiJen/VaxiJen.html'
        
    def get_data(self, source_filepath):
        epit_list = []
        score_list = []
        is_antigen = []
        
        #home page
        driver.get(self.base_url)
        sleep(3)
        
        #choosing file
        choose_file =  driver.find_element_by_xpath('/html/body/div/table/tbody/tr[4]/td[3]/form/table/tbody/tr[1]/td[2]/p/input')
        choose_file.send_keys(source_filepath)
        
        
        #selecting the target organism
        target_org = driver.find_element_by_xpath('/html/body/div/table/tbody/tr[4]/td[3]/form/table/tbody/tr[2]/td[2]/p/select/option[2]')
        target_org.click()
        
        #submit button
        submit = driver.find_element_by_xpath('/html/body/div/table/tbody/tr[4]/td[3]/form/table/tbody/tr[3]/td[2]/input[1]')
        submit.click()
        
        WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div/table/tbody/tr[3]/td[3]/font/b')))
            
        epit = driver.find_elements_by_xpath('//font[@face = "Courier New"]')
        score_tag = driver.find_elements_by_xpath('/html/body/div/table/tbody/tr[4]/td[3]/table/tbody/tr/td/b')
        anti = driver.find_elements_by_xpath('//td/b/font[@color = "#0000FF"]')
        
        #score is in <b> tag. There are variuos <b> tags. So we will use regex to check
        regex = '[+-]?[0-9]+\.[0-9]+'
        for i in range(0, len(score_tag)):
            if re.search(regex, score_tag[i].text):
                score_list.append(float(score_tag[i].text))
                
        for epi, is_ant in zip(epit, anti):
            epit_list.append(epi.text)
            is_antigen.append(is_ant.text)
            
        df = pd.DataFrame({"Epitope":epit_list, "Antigenicity":score_list, "Is_Antigen":is_antigen})
        return df
    
    
    def save_results(self, df, dest_filepath):
        df.to_csv(dest_filepath)
        
        
        
        

In [4]:
source_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\mhc1_epitopes_fasta.txt'
dest_filepath = r'C:\Users\Sahil\Omdena\Development of Vaccines for JC Virus\mhc1_epitopes_antigenicity.csv'

In [15]:
vA = Vaxijen_Antigenicity()
data = vA.get_data(source_filepath)
data

,Epitope,Antigenicity,Is_Antigen
0,ACFAVYTTK,1.2472,ANTIGEN
1,AVDTVLAKK,0.7139,ANTIGEN
2,AWLHCLLPK,-0.1332,NON-ANTIGEN
3,AYLRKCKEF,-0.0345,NON-ANTIGEN
4,FLICKGVNK,0.3501,NON-ANTIGEN
...,...,...,...
60,SQHSTPPKK,0.6622,ANTIGEN
61,TPHRHRVSA,0.1460,NON-ANTIGEN
62,VEESIQGGL,0.5589,ANTIGEN
63,YCFDCFRQW,0.1859,NON-ANTIGEN


In [17]:
vA.save_results(data, dest_filepath)